# Examine the dataset which chatgpt 4o mini selects

In [14]:
import torch
import pdb
model_a = '/data/nee7ne/effi_cot/saved_models/effi_cot/old_vanilla/sentence_transformer/model.pt'
model_b = '/data/nee7ne/effi_cot/saved_models/effi_cot/vanilla/gsm8k/sentence_transformer/model.pt'
def compare_model_parameters(model_a, model_b):
    """Compares the parameters of two PyTorch models."""

    # Check if the models have the same parameters
    # Load the models
    model_a = torch.load(model_a)
    model_b = torch.load(model_b)

    if set(model_a.keys()) != set(model_b.keys()):
        return False

    # Iterate through the parameters and compare them
    for param_name in model_a.keys():
        param_a = model_a[param_name]
        param_b = model_b[param_name]

        # Check if the parameters are equal
        print("model_a device", param_a.device)
        print("model_b device", param_b.device)
        param_a = param_a.cpu()
        param_b = param_b.cpu()
        if not torch.equal(param_a, param_b):
            # print where the parameters are not equal
            print(f"Parameters are not equal at {param_name}")
            print(f"Difference: {torch.abs(param_a - param_b).sum()}")
            # return False

    return True

compare_model_parameters(model_a, model_b)

model_a device cuda:0
model_b device cuda:2
Parameters are not equal at norm.weight
Difference: 1.5616649761795998e-05
model_a device cuda:0
model_b device cuda:2
Parameters are not equal at layers.0.self_attn.q_proj.weight
Difference: 0.011075451038777828
model_a device cuda:0
model_b device cuda:2
Parameters are not equal at layers.0.self_attn.k_proj.weight
Difference: 0.011102092452347279
model_a device cuda:0
model_b device cuda:2
Parameters are not equal at layers.0.self_attn.v_proj.weight
Difference: 0.008046792820096016
model_a device cuda:0
model_b device cuda:2
Parameters are not equal at layers.0.self_attn.o_proj.weight
Difference: 0.007584538776427507
model_a device cuda:0
model_b device cuda:2
Parameters are not equal at layers.0.mlp.gate_proj.weight
Difference: 0.026672109961509705
model_a device cuda:0
model_b device cuda:2
Parameters are not equal at layers.0.mlp.up_proj.weight
Difference: 0.02593364380300045
model_a device cuda:0
model_b device cuda:2
Parameters are not

KeyboardInterrupt: 